# Data Frame Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Frame Coding Quiz") \
    .getOrCreate()

In [29]:
log_data = spark.read.json("data/sparkify_log_small.json")

# Question 1

Which page did user id "" (empty string) NOT visit?

In [11]:
all_pages = log_data.select("page").dropDuplicates()
user_pages = log_data.select("page").filter(log_data.userId == '').dropDuplicates()

In [15]:
result = [page for page in all_pages if page not in user_pages]
result

[Row(page='Submit Downgrade'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [17]:
user_pages

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

# Question 3

How many female users do we have in the data set?

In [21]:
from pyspark.sql import functions as func
log_data.select("userId","gender").dropDuplicates()\
    .groupby("gender")\
    .agg(func.count("userId").alias("num_users"))\
    .orderBy(func.desc("num_users"))\
    .show()

+------+---------+
|gender|num_users|
+------+---------+
|     M|      501|
|     F|      462|
|  null|        1|
+------+---------+



# Question 4

How many songs were played from the most played artist?

In [23]:
log_data.select("artist")\
    .filter(log_data.page == 'NextSong')\
    .groupby("artist")\
    .agg(func.count("artist").alias("num_songs"))\
    .orderBy(func.desc("num_songs"))\
    .show(1)

+--------+---------+
|  artist|num_songs|
+--------+---------+
|Coldplay|       83|
+--------+---------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [34]:
from pyspark.sql import types
flag_homepage_visit = func.udf(lambda x: 1 if x=='Home' else 0, types.IntegerType())
log_data = log_data.withColumn("homepage_visit", flag_homepage_visit("page"))

In [37]:
log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- homepage_visit: integer (nullable = true)



In [40]:
from pyspark.sql import Window
windowval = Window.partitionBy("userId").orderBy("ts").rangeBetween(Window.unboundedPreceding, 0)
log_data = log_data.withColumn("phase", func.sum("homepage_visit").over(windowval))

In [41]:
log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- homepage_visit: integer (nullable = true)
 |-- phase: long (nullable = true)



In [45]:
log_data.select("userId","phase").filter("userId != ''")\
    .dropDuplicates().groupby("userId").agg(func.count("*").alias("num_phases"))\
    .orderBy(func.desc("num_phases"))\
    .show(10)

+------+----------+
|userId|num_phases|
+------+----------+
|  1647|        10|
|    58|         9|
|  2481|         8|
|  1290|         8|
|  1232|         8|
|  1959|         8|
|  1950|         7|
|  1865|         7|
|  1035|         7|
|   523|         6|
+------+----------+
only showing top 10 rows



In [61]:
song_flag = func.udf(lambda x: 1 if x == 'NextSong' else 0, types.IntegerType())
log_data = log_data.withColumn("song_flag", song_flag("page"))
songs_by_user_phase = log_data.select("userId","page","phase","song_flag")\
    .groupby("userId","phase")\
    .agg(func.sum("song_flag").alias("num_songs"),\
         func.max("phase").over(Window.partitionBy("userId")).alias("max_phase"))

In [62]:
songs_by_user_phase.filter("userId == 523").show(10)

+------+-----+---------+---------+
|userId|phase|num_songs|max_phase|
+------+-----+---------+---------+
|   523|    4|       17|        6|
|   523|    3|        0|        6|
|   523|    6|       16|        6|
|   523|    2|        8|        6|
|   523|    5|       27|        6|
|   523|    1|       22|        6|
+------+-----+---------+---------+



In [63]:
songs_by_user_phase.filter("phase > 0 and phase < max_phase").agg(func.mean("num_songs")).show()

+-----------------+
|   avg(num_songs)|
+-----------------+
|6.263862332695985|
+-----------------+



In [65]:
songs_by_user_phase.agg(func.mean("num_songs")).show()

+---------------+
| avg(num_songs)|
+---------------+
|5.0557238037553|
+---------------+



In [76]:
windowval2 = Window.partitionBy("userId").orderBy(func.desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
log_data2 = log_data.filter((log_data.page == 'NextSong') | (log_data.page == 'Home'))\
            .select("userId","page","ts","homepage_visit","song_flag")\
            .withColumn("phase", func.sum("homepage_visit").over(windowval2))
songs_by_user_phase2 = log_data2.select("userId","page","phase","song_flag")\
    .groupby("userId","phase")\
    .agg(func.sum("song_flag").alias("num_songs"),\
         func.max("phase").over(Window.partitionBy("userId")).alias("max_phase"))

In [77]:
songs_by_user_phase2.filter("userId == 523").orderBy("phase").show()

+------+-----+---------+---------+
|userId|phase|num_songs|max_phase|
+------+-----+---------+---------+
|   523|    0|       16|        6|
|   523|    1|       27|        6|
|   523|    2|       17|        6|
|   523|    3|        0|        6|
|   523|    4|        8|        6|
|   523|    5|       22|        6|
|   523|    6|        0|        6|
+------+-----+---------+---------+



In [82]:
songs_by_user_phase2.orderBy("userId").show(5)

+------+-----+---------+---------+
|userId|phase|num_songs|max_phase|
+------+-----+---------+---------+
|      |    5|        0|      187|
|      |    3|        0|      187|
|      |    4|        0|      187|
|      |    1|        0|      187|
|      |    2|        0|      187|
+------+-----+---------+---------+
only showing top 5 rows



In [83]:
songs_by_user_phase2.filter("num_songs > 0").agg(func.mean("num_songs")).show() #groupby("userId").count().orderBy("userId").show(10)

+-----------------+
|   avg(num_songs)|
+-----------------+
|6.898347107438017|
+-----------------+

